In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask
import os

: 

In [ ]:
def layer_alterator(
    raster_folder,
    vector_path,
    operation_rule,
    output_folder=None,
    value_range=(0, 1)
):
    """
    Modifies pixel values in multiple rasters based on vector mask attributes and specified rules.

    Parameters:
    - raster_folder (str): Folder containing rasters to be modified
    - vector_path (str): Path to vector mask (e.g. .gpkg, .shp)
    - operation_rule (str): operation types % or Reduction
    - output_folder (str, optional): Folder to save modified rasters. If None, saves to 'modified_<rastername>.tif'
    - value_range (tuple): (min, max) range for pixel clipping. Default is (0, 1)

    Returns:
    - None (writes modified raster files)
    """

    mask_gdf = gpd.read_file(vector_path)

    # List raster files
    raster_files = [f for f in os.listdir(raster_folder) if f.lower().endswith((".tif", ".tiff"))]

    for raster_file in raster_files:
        raster_path = os.path.join(raster_folder, raster_file)
        raster_key = os.path.splitext(raster_file)[0].lower()

        # Match raster name to rule
        operation = None
        attribute_field = None

        if not operation_rule:
            print(f"[SKIP] No operation rule for: {raster_file}")
            continue

        with rasterio.open(raster_path) as src:
            raster_data = src.read(1)
            transform = src.transform
            crs = src.crs
            dtype = raster_data.dtype

            for idx, row in mask_gdf.iterrows():
                if attribute_field not in row or row[attribute_field] is None:
                    continue

                geo = [row["geometry"]]
                value = row[attribute_field]

                out_image, _ = mask(src, geo, crop=False)
                out_image = out_image[0]

                polygon_mask, _ = mask(src, geo, crop=False)
                polygon_mask = polygon_mask[0]

                # Apply alteration
                if operation_rule == "increase_pct":
                    modified_image = out_image + (out_image * (value / 100.0))
                elif operation_rule == "reduction":
                    modified_image = out_image - value

                modified_image = np.clip(modified_image, *value_range)
                raster_data[polygon_mask > 0] = modified_image[polygon_mask > 0]

        # Prepare output path
        if output_folder:
            os.makedirs(output_folder, exist_ok=True)
            output_path = os.path.join(output_folder, raster_file)
        else:
            base_name = os.path.splitext(raster_file)[0]
            output_path = f"modified_{base_name}.tif"

        with rasterio.open(
            output_path, "w", driver="GTiff",
            height=raster_data.shape[0], width=raster_data.shape[1],
            count=1, dtype=dtype, crs=crs, transform=transform
        ) as dst:
            dst.write(raster_data, 1)

        print(f"[DONE] Modified {raster_file} -> {output_path}")


: 

In [ ]:
layer_alterator(
    vector_path="vector_mask_milan.gpkg",
    raster_folder="./Test Data",
    output_folder="output/modified_rasters",
    operation_rule="reduction",
    value_range=(0, 1)
)

[DONE] Modified grass.tif -> output/modified_rasters/grass.tif
[DONE] Modified asphalt.tif -> output/modified_rasters/asphalt.tif
